In [29]:
from datetime import datetime
import pandas as pd
import numpy as np
from dateutil.parser import parse

In [51]:
datestrs = ["08/08/1994", "24/08/1994", "07/09/1994", "21/09/1994"]
dates = []
for x in datestr:
    dates.append(parse(x, dayfirst=True))
dates

[datetime.datetime(1994, 8, 8, 0, 0),
 datetime.datetime(1994, 8, 24, 0, 0),
 datetime.datetime(1994, 9, 7, 0, 0),
 datetime.datetime(1994, 9, 21, 0, 0)]

In [52]:
pd.to_datetime(dates, dayfirst=True)

DatetimeIndex(['1994-08-08', '1994-08-24', '1994-09-07', '1994-09-21'], dtype='datetime64[ns]', freq=None)

In [54]:
ts = pd.Series(np.random.randn(4), index=dates)
ts

1994-08-08   -0.986803
1994-08-24    0.294990
1994-09-07   -0.576092
1994-09-21   -0.902798
dtype: float64

pandas.core.series.Series